In [1]:
import numpy as np
import pandas as pd
import pyMuellerMat
from pyMuellerMat.common_mm_functions import *
from pyMuellerMat import common_mms as cmm
from pyMuellerMat import MuellerMat
from scipy.optimize import minimize
import copy
from collections import defaultdict
from pathlib import Path

In [3]:
# make rebeccas csv reader compatible with my csv write function

def read_csv(file_path):
    """Takes a CSV file path containing "D_IMRANG", 
    "RET-ANG1", "single_sum", "norm_single_diff", "diff_std", "sum_std",
    and returns interleaved values, standard deviations, and configuration list.

    Parameters:
    -----------
    file_path : str or Path
        Path to the CSV.

    Returns:
    -----------
    interleaved_values : np.ndarray
        Interleaved values from "norm_single_diff" and "single_sum".
    interleaved_stds : np.ndarray
        Interleaved standard deviations from "diff_std" and "sum_std".
    configuration_list : list
        List of dictionaries containing configuration data for each row.
    """
    file_path = Path(file_path)
     
    # Read CSV file
    df = pd.read_csv(file_path)
    
    # Convert relevant columns to float (handling possible conversion errors)
    for col in ["RET-POS1", "D_IMRANG"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")  # Convert to float, set errors to NaN if not possible


    # Interleave values from "diff" and "sum"
    interleaved_values = np.ravel(np.column_stack((df["norm_single_diff"].values, df["single_sum"].values)))

    # Interleave values from "diff_std" and "sum_std"
    interleaved_stds = np.ravel(np.column_stack((df["diff_std"].values, df["sum_std"].values)))

    # Convert each row's values into a list of two-element lists
    configuration_list = []
    for index, row in df.iterrows():
        # Extracting values from relevant columns
        hwp_theta = row["RET-POS1"]
        imr_theta = row["D_IMRANG"]

        # Building dictionary
        row_data = {
            "hwp": {"theta": hwp_theta},
            "image_rotator": {"theta": imr_theta}
        }

        # Append two configurations for diff and sum
        configuration_list.append(row_data)

    return interleaved_values, interleaved_stds, configuration_list